# PDF to Medium Article Generator with Persistent Vector Store

This notebook creates a system that:
1. Reads PDF files from a specified folder
2. Persistently stores their content in a Chroma vector store
3. Generates technical Medium articles through a Gradio chat interface
4. Allows resuming processing from previous state
5. Formats articles in active voice, targeting recruiters and managers

Key Features:
- Persistent vector store using SQLite backend
- Progress tracking and state management
- Chunk-based processing with automatic saves
- Resume capability for interrupted processing

In [1]:
# Install required packages
!pip install langchain langchain-community chromadb pypdf gradio requests numpy tqdm

In [1]:
import os
import time
from datetime import datetime
from pathlib import Path
import json
import hashlib
import math
import requests
import numpy as np
from tqdm.notebook import tqdm

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import gradio as gr

c:\Users\kstvk\anaconda3\envs\ai-engineering\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


# Vector Store Configuration and Helper Functions

We'll set up:
1. Directory configuration for persistent storage
2. Helper functions for time formatting and progress tracking
3. Vector store initialization and management functions
4. Processing progress tracking and state management

In [2]:
# Helper function for time formatting
def format_time(seconds):
    """Convert seconds to human readable time format"""
    if seconds < 60:
        return f"{seconds:.1f} seconds"
    elif seconds < 3600:
        minutes = seconds / 60
        return f"{minutes:.1f} minutes"
    else:
        hours = seconds / 3600
        return f"{hours:.1f} hours"

# Vector store configuration
PERSIST_DIRECTORY = os.path.join(os.getcwd(), "vector_store")
PROGRESS_FILE = os.path.join(PERSIST_DIRECTORY, "processing_progress.json")

def verify_ollama_connection():
    """Verify that Ollama is running and accessible."""
    try:
        response = requests.get("http://localhost:11434/api/tags")
        if response.status_code == 200:
            print("✓ Ollama connection verified")
            return True
        else:
            print("✗ Ollama is not responding correctly")
            return False
    except requests.exceptions.RequestException as e:
        print("✗ Could not connect to Ollama. Is it running?")
        print(f"Error: {str(e)}")
        return False

def test_embeddings():
    """Test that embeddings are working correctly."""
    try:
        embeddings = OllamaEmbeddings(model="llama3")
        test_text = "This is a test sentence."
        result = embeddings.embed_query(test_text)
        
        if isinstance(result, list) and len(result) > 0 and all(isinstance(x, float) for x in result):
            print("✓ Embeddings test successful")
            return True
        else:
            print("✗ Invalid embedding output format")
            return False
    except Exception as e:
        print("✗ Embeddings test failed")
        print(f"Error: {str(e)}")
        return False

def save_processing_progress(processed_files):
    """Save the list of processed files and their hashes."""
    os.makedirs(PERSIST_DIRECTORY, exist_ok=True)
    with open(PROGRESS_FILE, "w") as f:
        json.dump(processed_files, f)

def load_processing_progress():
    """Load the list of previously processed files."""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, "r") as f:
            return json.load(f)
    return {}

def get_file_hash(file_path):
    """Calculate hash of file for tracking changes."""
    with open(file_path, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()

def initialize_vector_store():
    """Initialize or load existing vector store."""
    os.makedirs(PERSIST_DIRECTORY, exist_ok=True)
    embeddings = OllamaEmbeddings(model="llama3")
    
    if os.path.exists(os.path.join(PERSIST_DIRECTORY, "chroma.sqlite3")):
        print("Loading existing vector store...")
        return Chroma(
            persist_directory=PERSIST_DIRECTORY,
            embedding_function=embeddings
        )
    else:
        print("Creating new vector store...")
        return Chroma(
            persist_directory=PERSIST_DIRECTORY,
            embedding_function=embeddings
        )

def save_vector_store(vectorstore):
    """Save vector store to disk."""
    print("Saving vector store...")
    vectorstore.persist()
    print(f"Vector store saved to {PERSIST_DIRECTORY}")

def clear_vector_store():
    """Clear the vector store and processing history."""
    import shutil
    if os.path.exists(PERSIST_DIRECTORY):
        shutil.rmtree(PERSIST_DIRECTORY)
        os.makedirs(PERSIST_DIRECTORY)
        print("Vector store cleared successfully")

def get_vector_store_stats():
    """Get statistics about the vector store."""
    if os.path.exists(os.path.join(PERSIST_DIRECTORY, "chroma.sqlite3")):
        vectorstore = initialize_vector_store()
        collection = vectorstore._collection
        stats = {
            "total_documents": collection.count(),
            "persist_directory": PERSIST_DIRECTORY,
            "processed_files": len(load_processing_progress())
        }
        return stats
    return {
        "total_documents": 0, 
        "persist_directory": PERSIST_DIRECTORY, 
        "processed_files": 0
    }

# Document Processing and Vector Store Population

Now we'll implement the core document processing functionality:
1. Load PDFs from the specified directory
2. Split documents into chunks
3. Process and store embeddings with progress tracking
4. Save state at regular intervals

In [14]:
def load_pdfs(directory):
    """Load all PDFs from the specified directory."""
    pdf_files = list(Path(directory).glob("*.pdf"))
    if not pdf_files:
        print("No PDF files found in the specified directory.")
        return []
    
    print(f"\nLoading {len(pdf_files)} PDF files...")
    documents = []
    for pdf_path in pdf_files:
        try:
            loader = PyPDFLoader(str(pdf_path))
            documents.extend(loader.load())
            print(f"✓ Loaded {pdf_path.name}")
        except Exception as e:
            print(f"✗ Error loading {pdf_path.name}: {str(e)}")
    return documents

def process_documents(documents, vectorstore=None):
    """Split documents and create/update vector store with progress tracking."""
    if not documents:
        print("No documents to process.")
        return initialize_vector_store()
    
    # Verify Ollama connection and embeddings before processing
    if not verify_ollama_connection() or not test_embeddings():
        raise RuntimeError("Failed to initialize Ollama and embeddings. Please check the error messages above.")
    
    # Initialize or load vector store
    if vectorstore is None:
        vectorstore = initialize_vector_store()
    
    # Load processing progress
    processed_files = load_processing_progress()
    
    # Initialize timing and progress tracking
    start_time = time.time()
    total_docs = len(documents)
    print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Starting processing of {total_docs} documents")
    
    # Text splitter with optimized chunk size
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
    )
    
    # Estimate chunks and time
    sample_size = min(5, len(documents))
    sample_chunks = sum(len(text_splitter.split_text(doc.page_content)) for doc in documents[:sample_size])
    estimated_total_chunks = math.ceil((sample_chunks / sample_size) * len(documents))
    print(f"Estimated total chunks: {estimated_total_chunks} (based on {sample_size} document sample)")
    
    # Process documents with progress tracking
    processed_chunks = 0
    save_interval = 30  # Save every 30 chunks for better persistence
    last_saved_chunk = 0
    
    for i, doc in enumerate(documents, 1):
        # Check if document was already processed
        if doc.metadata.get('source') in processed_files:
            print(f"Skipping already processed document: {doc.metadata.get('source')}")
            continue
        print(f"\nProcessing document {i}/{total_docs}: {doc.metadata.get('source', 'Unknown Source')}")
        chunk_start = time.time()
        texts = text_splitter.split_documents([doc])
        processed_chunks += len(texts)
        
        try:
            # Add to vector store with retry logic
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    vectorstore.add_documents(texts)
                    
                    # Save progress periodically
                    if processed_chunks > last_saved_chunk + save_interval:
                        save_vector_store(vectorstore)
                        print(f"\nProgress saved. Total chunks processed: {processed_chunks}")
                        last_saved_chunk = processed_chunks
                    
                    break
                except ValueError as e:
                    if attempt == max_retries - 1:
                        raise
                    print(f"Retry {attempt + 1}/{max_retries} - Waiting 2 seconds before retry...")
                    time.sleep(2)
        
            # Calculate progress and estimates
            chunk_time = time.time() - chunk_start
            elapsed_total = time.time() - start_time
            avg_time_per_chunk = elapsed_total / processed_chunks
            estimated_remaining_chunks = estimated_total_chunks - processed_chunks
            estimated_remaining_time = estimated_remaining_chunks * avg_time_per_chunk
            
            print(f"\nProgress: Document {i}/{total_docs}")
            print(f"Chunks in this document: {len(texts)}")
            print(f"Total chunks processed: {processed_chunks}/{estimated_total_chunks}")
            print(f"Time for this document: {format_time(chunk_time)}")
            print(f"Estimated remaining time: {format_time(estimated_remaining_time)}")
        
        except Exception as e:
            print(f"\nERROR processing document {i}: {str(e)}")
            print("Continuing with next document...")
            continue
    
    # Final save
    save_vector_store(vectorstore)
    processed_files[doc.metadata.get('source')] = get_file_hash(doc.metadata.get('source'))
    save_processing_progress(processed_files)
    
    # Final statistics
    total_time = time.time() - start_time
    print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Processing completed:")
    print(f"Total documents processed: {total_docs}")
    print(f"Total chunks created: {processed_chunks}")
    print(f"Total processing time: {format_time(total_time)}")
    print(f"Average time per chunk: {(total_time/processed_chunks):.2f} seconds")
    
    # Show vector store stats
    stats = get_vector_store_stats()
    print("\nVector Store Statistics:")
    print(f"Total documents in store: {stats['total_documents']}")
    print(f"Total processed files: {stats['processed_files']}")
    print(f"Store location: {stats['persist_directory']}")
    
    return vectorstore

# Article Generation with LLM Chain

We'll set up the article generation pipeline:
1. Create a prompt template for technical articles
2. Initialize the LLM chain with Ollama
3. Create a function to generate articles from vector store content
4. Add a Gradio interface for easy interaction

In [15]:
# Setup the article generation chain
article_template = """
You are an expert technical writer and software engineer creating a Medium article to showcase your skills in AI and ML.
Topic: {topic}
Retrieved Content: {context}

Write a 1200-word technical article that:
1. Uses active voice throughout
2. Explains technical concepts in business-friendly language
3. Highlights practical applications and business value
4. Includes relevant examples from the source material
5. Structures content with clear headings and subheadings

Article:
"""

article_prompt = PromptTemplate(
    input_variables=["topic", "context"],
    template=article_template
)

# Initialize Ollama with llama3 model
llm = Ollama(model="llama3")
article_chain = LLMChain(llm=llm, prompt=article_prompt)

def generate_article(topic, vectorstore):
    """Generate a Medium article based on the topic and vector store content."""
    # Search the vector store for relevant content
    results = vectorstore.similarity_search(topic, k=5)
    context = "\n".join([doc.page_content for doc in results])
    
    # Generate the article
    article = article_chain.run(topic=topic, context=context)
    return article

def gradio_interface(topic):
    """Gradio interface function for article generation."""
    try:
        if not any(Path(pdf_directory).glob("*.pdf")):
            return "Error: No PDF files found in the 'pdfs' directory. Please add some PDF files first."
        
        # Initialize vector store
        vectorstore = initialize_vector_store()
        if not vectorstore:
            return "Error: Could not initialize vector store. Please check if the store exists and contains documents."
        
        article = generate_article(topic, vectorstore)
        return article
    except Exception as e:
        return f"Error generating article: {str(e)}"

In [ ]:
# Directory setup and initial processing
pdf_directory = "pdfs"
os.makedirs(pdf_directory, exist_ok=True)

# Process PDFs if they exist
if any(Path(pdf_directory).glob("*.pdf")):
    print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Starting PDF processing pipeline")
    
    # Verify Ollama setup first
    if verify_ollama_connection() and test_embeddings():
        # Show existing vector store stats if any
        print("\nCurrent Vector Store Status:")
        stats = get_vector_store_stats()
        print(f"Total documents in store: {stats['total_documents']}")
        print(f"Total processed files: {stats['processed_files']}")
        
        # Process documents
        documents = load_pdfs(pdf_directory)
        vectorstore = process_documents(documents)
        print(f"[{datetime.now().strftime('%H:%M:%S')}] Vector store creation completed")
    else:
        print("Setup verification failed. Please check the error messages above.")
else:
    print("No PDF files found in the 'pdfs' directory. Please add some PDF files to start processing.")

# Create and launch the Gradio interface



[19:17:56] Starting PDF processing pipeline
✓ Ollama connection verified
✓ Embeddings test successful

Current Vector Store Status:
Loading existing vector store...
Total documents in store: 1453
Total processed files: 0

Loading 1 PDF files...
✓ Loaded Hands-On Generative AI with Transformers and Diffusion.pdf
✓ Ollama connection verified
✓ Embeddings test successful
Loading existing vector store...

[19:18:30] Starting processing of 819 documents
Estimated total chunks: 328 (based on 5 document sample)

Processing document 1/819: pdfs\Hands-On Generative AI with Transformers and Diffusion.pdf
Retry 1/3 - Waiting 2 seconds before retry...
Retry 2/3 - Waiting 2 seconds before retry...

ERROR processing document 1: Expected Embeddings to be non-empty list or numpy array, got [] in upsert.
Continuing with next document...

Processing document 2/819: pdfs\Hands-On Generative AI with Transformers and Diffusion.pdf
Retry 1/3 - Waiting 2 seconds before retry...
Retry 2/3 - Waiting 2 seconds

C:\Users\kstvk\AppData\Local\Temp\ipykernel_28236\2443473163.py:89: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()



Progress: Document 27/819
Chunks in this document: 2
Total chunks processed: 33/328
Time for this document: 5.6 seconds
Estimated remaining time: 15.6 minutes

Processing document 28/819: pdfs\Hands-On Generative AI with Transformers and Diffusion.pdf

Progress: Document 28/819
Chunks in this document: 1
Total chunks processed: 34/328
Time for this document: 3.0 seconds
Estimated remaining time: 15.5 minutes

Processing document 29/819: pdfs\Hands-On Generative AI with Transformers and Diffusion.pdf

Progress: Document 29/819
Chunks in this document: 1
Total chunks processed: 35/328
Time for this document: 2.8 seconds
Estimated remaining time: 15.4 minutes

Processing document 30/819: pdfs\Hands-On Generative AI with Transformers and Diffusion.pdf

Progress: Document 30/819
Chunks in this document: 2
Total chunks processed: 37/328
Time for this document: 5.4 seconds
Estimated remaining time: 15.2 minutes

Processing document 31/819: pdfs\Hands-On Generative AI with Transformers and Di

In [7]:
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Enter the topic for your technical article...",
        label="Article Topic"
    ),
    outputs=gr.Textbox(
        lines=20,
        label="Generated Article"
    ),
    title="Technical Article Generator",
    description="Generate a 1000-word technical article from your PDF content, targeted at recruiters and managers."
)

# Launch the interface
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Loading existing vector store...


C:\Users\kstvk\AppData\Local\Temp\ipykernel_51984\2794170847.py:33: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  article = article_chain.run(topic=topic, context=context)


In [ ]:
iface.close()  # Close the interface when done

# How to Use

1. Create a directory named `pdfs` in the same location as this notebook
2. Place your PDF files in the `pdfs` directory
3. Install Ollama on your system and start the Ollama service
4. Pull the llama3 model using: `ollama pull llama3`
5. Run all cells in this notebook
6. Use the Gradio interface to:
   - Enter your desired article topic
   - Click submit to generate the article
   - Copy the generated article and paste it into Medium

The system will:
- Automatically load and process PDFs from the `pdfs` directory
- Store embeddings persistently in the `vector_store` directory
- Save progress every 30 chunks during processing
- Allow resuming from previous state if interrupted
- Generate articles using the stored knowledge

Notes:
- Make sure Ollama is running before using this notebook
- The vector store is persistent and will be reused across sessions
- You can clear the vector store using the `clear_vector_store()` function if needed
- Progress tracking prevents reprocessing of already processed files

Example Usage:
```python
# Get current vector store statistics
stats = get_vector_store_stats()
print(f"Documents in store: {stats['total_documents']}")

# Clear vector store and start fresh
clear_vector_store()

# Process new documents
documents = load_pdfs(pdf_directory)
vectorstore = process_documents(documents)

# Save explicitly if needed
save_vector_store(vectorstore)
```